In [1]:
import cv2
import os
import numpy as np

class AdvancedCannyDetector:
    def __init__(self):
        pass

    def reduce_noise(self, image, kernel_size=5):
        """降噪處理: 使用高斯濾波器去除雜訊"""
        return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

    def color_processing(self, image):
        """顏色信息處理: 轉換至HSV空間並提取Value通道"""
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(hsv)
        return v

    def multi_scale_edges(self, image, scales=[1, 2, 4]):
        """多尺度邊緣檢測: 在不同尺度下檢測邊緣並融合"""
        edges_final = np.zeros(image.shape, dtype=np.uint8)
        
        for scale in scales:
            # 重新縮放圖像
            scaled = cv2.resize(image, None, fx=1/scale, fy=1/scale)
            # 檢測邊緣
            edges = cv2.Canny(scaled, 50, 150)
            # 縮放回原始大小
            edges = cv2.resize(edges, (image.shape[1], image.shape[0]))
            # 融合邊緣
            edges_final = cv2.bitwise_or(edges_final, edges)
        
        return edges_final

    def adaptive_threshold(self, image):
        """自適應閾值計算: 使用圖像統計特徵決定閾值"""
        mean = np.mean(image)
        std = np.std(image)
        
        lower = int(max(0, (1.0 - 0.33) * mean))
        upper = int(min(255, (1.0 + 0.33) * mean))
        
        return lower, upper

    def edge_thinning(self, edges):
        """邊緣細化: 使用形態學操作細化邊緣"""
        kernel = np.ones((3,3), np.uint8)
        thinned = cv2.morphologyEx(edges, cv2.MORPH_OPEN, kernel)
        return thinned

    def high_frequency_enhancement(self, image):
        """高頻增強: 使用unsharp masking技術增強高頻細節"""
        gaussian = cv2.GaussianBlur(image, (5,5), 2.0)
        enhanced = cv2.addWeighted(image, 1.5, gaussian, -0.5, 0)
        return enhanced

    def detect_edges(self, image):
        """整合所有步驟的主要處理函數"""
        # 1. 降噪處理
        denoised = self.reduce_noise(image)
        
        # 2. 顏色處理
        value_channel = self.color_processing(denoised)
        
        # 3. 高頻增強
        enhanced = self.high_frequency_enhancement(value_channel)
        
        # 4. 計算自適應閾值
        low_threshold, high_threshold = self.adaptive_threshold(enhanced)
        
        # 5. 多尺度邊緣檢測
        edges = self.multi_scale_edges(enhanced)
        
        # 6. 邊緣細化
        final_edges = self.edge_thinning(edges)
        
        return final_edges


def process_images_in_folder(input_folder, output_folder):
    """批量處理資料夾內的圖片"""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    detector = AdvancedCannyDetector()

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            # 讀取圖片
            image = cv2.imread(input_path)
            if image is None:
                print(f"無法讀取圖片: {input_path}")
                continue

            # 偵測邊緣
            edges = detector.detect_edges(image)

            # 儲存結果
            cv2.imwrite(output_path, edges)
            print(f"已處理: {input_path} -> {output_path}")


# 使用範例
input_folder = "/home/yuchi/Canny_Detection/crackforest/Images"  # 輸入資料夾
output_folder = "/home/yuchi/Canny_Detection/crackforest/Output"  # 輸出資料夾
process_images_in_folder(input_folder, output_folder)
